## More Lions is All You Need

running scoring and voting using `llama2` model via `ollama` service

In [1]:
import lionagi as li

from IPython.display import Markdown
from timeit import default_timer as timer

In [2]:
system = "you are a helpful assistant."
instruct = "think step by step, concisely yet comprehensively explain the balance sheet. return around 200 words"

The goal is to find the best answer from 5 generations of the same questions

In [3]:
from lionagi import Services, Session

In [4]:
durations = []

async def explain(i):
    service = Services.Ollama(model='llama2')
    session = Session(system=system, service=service)

    # use timed call to get the runtime of the given function
    out, time_ = await li.tcall(
        func=session.chat, instruction=instruct, timing=True
    )
    
    durations.append(time_)
    return session

In [5]:
start_time = timer()

In [6]:
writer_sessions = await li.alcall([i for i in range(5)], explain)

In [7]:
run_time = timer() - start_time
print(f"\nClock time: {run_time:.2f} seconds")


Clock time: 51.93 seconds


In [8]:
import numpy as np

run_describe = f"""
Average runtime: {np.average(durations):.2f} seconds\n
Total runtime: {np.sum(durations):.2f} seconds\n
Total number of runs: {len(durations)}
"""
print(run_describe)


Average runtime: 35.55 seconds

Total runtime: 177.74 seconds

Total number of runs: 5



In [9]:
print(f"speed up { np.sum(durations) / float(run_time): .02f}")

speed up  3.42


### Get a group of critics

In [10]:
system = "as a highly critical professor, score an answer, only return a positive integer"
instruction = "meticulously and critically give a score between 0-10 on the following explaination of balance sheet."

In [11]:
durations = []
results = []

async def score(context):
    service = Services.Ollama(model='llama2')
    
    # set up a function for one scorer to evaluate
    async def _inner(i):
        session = Session(system=system, service=service)
        
        out_, time_ = await li.tcall(
            func=session.chat, instruction=instruction, context=context, timing=True)
        
        durations.append(time_)
        return out_
        
    scores = await li.alcall([i for i in range(5)], _inner)        # run 5 scorers in parallel 
    
    out = {
        "context": context,
        "scores": scores
    }
    
    results.append(out)
    
    return (scores, context)

In [12]:
# get the responses from all previous sessions

responses = li.lcall(writer_sessions, lambda s: s.last_response_content['response'])

this is 25 runs in total across all responses,
so will be pretty slow

to speed up, reduce the number of responses


In [13]:
start_time = timer()

In [14]:
outputs = await li.alcall(responses, score)

In [15]:
run_time = timer() - start_time
print(f"\nClock time: {run_time:.2f} seconds")


Clock time: 225.78 seconds


In [16]:
scores = [o[0] for o in outputs]
contexts = [o[1] for o in outputs]

In [17]:
run_time = f"""
Average runtime: {np.average(durations):.2f} seconds\n
Total runtime: {np.sum(durations):.2f} seconds\n
Total number of runs: {len(durations)}
"""
print(run_time)


Average runtime: 152.49 seconds

Total runtime: 3812.32 seconds

Total number of runs: 25



In [18]:
avgs = []
for i in scores:
    z = []
    for j in i:
        try:
            a = li.str_to_num(j)
            z.append(a)
        except:
            pass
    avgs.append(np.average(z))

print(avgs)

[7.2, 6.4, 8.0, 6.8, 8.4]


In [19]:
high_index = np.where(np.array(avgs) == max(avgs))
low_index = np.where(np.array(avgs) == min(avgs))

In [20]:
high_index

(array([4]),)

In [21]:
low_index

(array([1]),)

### High Score

In [22]:
print(f"High score: {max(avgs)}")
Markdown(contexts[4])

High score: 8.4


Of course! Here's a step-by-step explanation of a balance sheet:

A balance sheet is a financial statement that provides a snapshot of a company's financial position at a specific point in time, usually at the end of an accounting period. It lists the company's assets, liabilities, and equity, and shows how these components relate to each other.

Assets: These are the resources owned by the company that can be converted into cash or used to generate revenue. Examples include cash, accounts receivable, inventory, property, plant, and equipment.

Liabilities: These are the debts or obligations owed by the company to others. Examples include loans, accounts payable, taxes payable, and bonds.

Equity: This represents the ownership interest in the company, calculated as the total value of assets minus the total value of liabilities. Equity can also refer to shareholder equity, which is the residual interest after deducting the total value of liabilities from the total value of assets.

The balance sheet is important because it provides a comprehensive picture of a company's financial position at a given point in time. It helps investors, creditors, and other stakeholders evaluate the company's financial health and make informed decisions about investing, lending, or doing business with the company.

The balance sheet is structured into two main sections: assets and liabilities. Assets are listed on the left side of the balance sheet, while liabilities and equity are listed on the right side. Within each section, the items are arranged in order of their liquidity or ease of conversion into cash.

The balance sheet is prepared using generally accepted accounting principles (GAAP) or international financial reporting standards (IFRS), which provide guidelines for how to classify and present financial information. The balance sheet is used in conjunction with other financial statements, such as the income statement and statement of cash flows, to provide a complete picture of a company's financial performance and position over time.

In summary, the balance sheet provides a snapshot of a company's financial position at a specific point in time, showing its assets, liabilities, and equity. It is an important tool for investors, creditors, and other stakeholders to evaluate a company's financial health and make informed decisions.

### Low score

In [23]:
print(f"Low score: {min(avgs)}")
Markdown(contexts[1])

Low score: 6.4



As a helpful assistant, I'd be happy to explain the balance sheet in a clear and concise manner! Here it goes:

The balance sheet is a financial statement that provides a snapshot of a company's assets, liabilities, and equity at a specific point in time. It helps investors and stakeholders understand the company's financial position and make informed decisions about investing or lending to the company.

The balance sheet consists of two main sections: assets and liabilities. Assets are anything that the company owns or has control over, such as cash, inventory, property, and equipment. Liabilities are debts or obligations that the company owes to others, such as loans, accounts payable, and taxes owed.

The key to understanding the balance sheet is to recognize that assets must equal liabilities plus equity. This is known as the "accounting equation" or "balance sheet formula." In other words, the value of a company's assets must equal the sum of its liabilities and equity. If the value of assets exceeds liabilities and equity, the company has net assets or equity, which can be distributed to shareholders as dividends or used for investment purposes.

Now, let me break it down step by step:

1. Assets: These are the resources owned or controlled by the company, including cash, inventory, property, and equipment.
2. Liabilities: These are debts or obligations that the company owes to others, such as loans, accounts payable, and taxes owed.
3. Equity: This represents the ownership interest in the company, consisting of shareholder equity and retained earnings.
4. Assets = Liabilities + Equity: This is the basic accounting equation that balances the company's financial statements.

In summary, the balance sheet provides a comprehensive picture of a company's financial position by listing its assets, liabilities, and equity. By understanding this equation, investors and stakeholders can evaluate a company's financial health and make informed decisions about investing or lending to the company.

### Others

In [24]:
print(f"High score: {avgs[0]}")
Markdown(contexts[0])

High score: 7.2



As a helpful assistant, I'd be delighted to explain the balance sheet in a clear and concise manner! Here's a step-by-step guide:

1. Definition: A balance sheet is a financial statement that provides a snapshot of a company's financial position at a specific point in time. It lists the company's assets, liabilities, and equity.
2. Assets: The assets section includes all the resources owned by the company, such as cash, accounts receivable, inventory, property, plant, and equipment. These assets are classified into current assets (assets that can be converted into cash within one year) and non-current assets (assets that cannot be converted into cash within one year).
3. Liabilities: The liabilities section includes all the debts or obligations owed by the company, such as accounts payable, loans, and long-term debt. These liabilities are also classified into current liabilities (liabilities that can be paid off within one year) and non-current liabilities (liabilities that cannot be paid off within one year).
4. Equity: The equity section represents the ownership interest in the company, including common stock, preferred stock, and retained earnings. This section shows the amount of money invested in the business by the shareholders.
5. Classifications: Assets are classified into current or non-current based on their liquidity. Liabilities are also classified into current or non-current based on their maturity date.
6. Calculation: The balance sheet shows the calculated values of total assets, total liabilities, and total equity. These values are obtained by adding up the respective accounts in the assets, liabilities, and equity sections.
7. Interpretation: The balance sheet provides valuable information to stakeholders, such as investors, creditors, and managers, about the financial position and performance of the company. By analyzing the balance sheet, they can assess the company's solvency, profitability, and liquidity.
8. Importance: The balance sheet is an essential tool for businesses to manage their finances, make informed decisions, and report their financial performance to stakeholders. It provides a comprehensive view of the company's financial position and helps identify areas for improvement.

In conclusion, the balance sheet is a critical financial statement that provides a detailed overview of a company's assets, liabilities, and equity. By understanding the components of the balance sheet and how to interpret it, stakeholders can gain valuable insights into the company's financial position and make informed decisions.

In [25]:
print(f"High score: {avgs[2]}")
Markdown(contexts[2])

High score: 8.0



As a helpful assistant, I'd be delighted to provide you with a clear and concise explanation of a balance sheet! A balance sheet is a financial statement that shows the financial position of a company at a particular point in time, including its assets, liabilities, and equity. Here's how it works:

Assets: These are the resources owned by the company that can be converted into cash or used to generate revenue. Examples include cash, accounts receivable, inventory, property, plant, and equipment (PP&E).

Liabilities: These are the debts or obligations owed by the company at a given time. Examples include loans, accounts payable, taxes payable, and long-term debt.

Equity: This represents the ownership interest in the company. It includes shareholders' equity (common stock and retained earnings) and any additional paid-in capital.

The balance sheet is structured as a tally of assets equaling the sum of liabilities plus equity (A = L + E). This equation ensures that the total value of a company's assets equals the amount of its liabilities plus the ownership interest held by shareholders.

When interpreting a balance sheet, investors and analysts focus on several key ratios, such as:

* Debt-to-Equity Ratio: This compares a company's total liabilities to its shareholder equity. A lower ratio generally indicates better financial health.
* Current Ratio: This measures the ability of a company to pay its short-term debts by dividing current assets into current liabilities. A higher ratio is generally considered more favorable.
* Return on Equity (ROE): This represents how effectively a company uses its equity to generate profits. A higher ROE indicates better financial performance.

By analyzing these ratios and the balance sheet components, investors can gain insights into a company's financial health and make informed investment decisions.

In [26]:
print(f"High score: {avgs[3]}")
Markdown(contexts[3])

High score: 6.8



As a helpful assistant, I'm glad to provide you with a clear and concise explanation of the balance sheet. The balance sheet is a financial statement that provides a snapshot of a company's financial position at a specific point in time, typically at the end of an accounting period. It reports on the company's assets, liabilities, and equity.

Assets: These are the resources owned by the company that can be converted into cash or used to generate revenue. Examples include cash, accounts receivable, inventory, property, plant, and equipment.

Liabilities: These are the debts or obligations of the company that must be paid using the company's assets or profits. Examples include loans, accounts payable, taxes owed, and long-term debt.

Equity: This represents the ownership interest in the company, including shares held by shareholders and retained earnings (profits). Equity is the residual interest after deducting liabilities from assets.

The balance sheet is important because it provides stakeholders with a comprehensive view of the company's financial position at a given point in time. It helps investors, creditors, and other stakeholders evaluate the company's financial health and make informed decisions. The balance sheet also serves as a framework for creating financial statements such as the income statement and cash flow statement.

In conclusion, the balance sheet is a critical financial statement that provides valuable information on a company's assets, liabilities, and equity. It helps stakeholders understand the company's financial position and make informed decisions.